In [15]:
# Imports
import numpy.typing as npt
import numpy as np
import pandas as df
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [20]:
# Linear Function for use in Curve Fit
def linearFunc(x: float, a: float) -> float:
    return a * x

# Define Dataset Class
class dataSet:
    partNum: int
    amps: npt.NDArray[np.float64] = np.array([0]) # [A]
    meanV: npt.NDArray[np.float64] = np.array([0]) # [mV]

    resist: float # [Ohm * m]
    cond: float # [S/m]

    # Units in [mm] and [mm^2]
    length: float
    side1: float
    side2: float
    area: float

    # Methods :
    def __init__(self, prt: npt.NDArray[np.float64]) -> None:
        self.partNum = int(prt[0][0][-1]) # Get Part Num
        prt = np.delete(prt, (0), axis=0) # Remove Header
        len = prt.shape[0]

        curr: int = 0
        count: int = 0
        for i in range(0,len):
            prt[i][0] = float(prt[i][0]) # Make everything floats
            if self.amps[curr] != prt[i][0] :
                if count > 0 :
                    self.meanV[curr] /= count
                self.amps = np.append(self.amps,prt[i][0])
                self.meanV = np.append(self.meanV,0.0)
                curr += 1
                count = 0
            self.meanV[curr] += abs(prt[i][1] - prt[i][2]) # Calculate the actual ΔV from Measurement and Baseline
            count += 1
        else: # Divide last meanV
            self.meanV[curr] /= count
        self.amps = np.delete(self.amps,0)
        self.meanV = np.delete(self.meanV,0)

    def setSize(self, length:float, side1: float, side2: float) -> None:
        self.length = length
        self.side1 = side1
        self.side2 = side2
        self.area = side1 * side2

    def calcResistance(self) -> None:
        popt: list[float] = curve_fit(linearFunc, self.amps, self.meanV)[0]
        print(popt)
        R = 1

        self.resist = R * self.area / self.length
        self.cond = 1 / self.resist


# Read Data
part1 = dataSet(df.read_csv("part1.csv").to_numpy())
part2_1 = dataSet(df.read_csv("part2_1.csv").to_numpy())
part2_2 = dataSet(df.read_csv("part2_2.csv").to_numpy())
part3 = dataSet(df.read_csv("part3.csv").to_numpy())
# print(part1.amps.tolist())
# print(part1.meanV.tolist())

part1.setSize(67.58 - 40.98, 2.77, 3.13)

part1.calcResistance()

[1.0862225108]


AttributeError: 'dataSet' object has no attribute 'area'